# In this notebook I will perform some Datawrangling using Python API of Spark

**The data is obtained from MovieLens can be found here https://grouplens.org/datasets/movielens/25m/**

Import findspark to run spark on local mode in windows

In [1]:
import findspark

In [2]:
findspark.init()

Import Spark Session from Pyspark.sql to create well a Spark Session

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Movie lens data wrangling").getOrCreate()

Now that we have the spark session up and running we can read the files, the files are in csv format (comma separated) where comma are escaped in double quotes. The files are encoded in UTF-8.

In [10]:
! dir movielens\ml-25m

 El volumen de la unidad C es Windows
 El n£mero de serie del volumen es: 55D7-D790

 Directorio de C:\Users\Usuario\Desktop\Pyspark-demo\Data wrangling\movielens\ml-25m

12/10/2020  12:23    <DIR>          .
12/10/2020  12:23    <DIR>          ..
12/10/2020  12:23       435.164.157 genome-scores.csv
12/10/2020  12:23            18.103 genome-tags.csv
12/10/2020  12:23         1.368.578 links.csv
12/10/2020  12:23         3.038.099 movies.csv
12/10/2020  12:23       678.260.987 ratings.csv
12/10/2020  12:23            10.460 README.txt
12/10/2020  12:23        38.810.332 tags.csv
               7 archivos  1.156.670.716 bytes
               2 dirs  787.255.750.656 bytes libres


**We can see that we have 6 csv files at our disposal**

I am going to start by reading ratings.csv 

In [17]:
ratings = spark.read.format("csv").option("delimiter", ","
                                         ).option("ESCAPE quote", '"'
                                                 ). option("header",True
                                                          ).option("encoding", "UTF-8").load("movielens/ml-25m/ratings.csv"
                                                                                            ).coalesce(4)

I am going to partition the file into 4 chunks to improve performance (pay attention the last chained command **coalesce**), but when you are working on a cluster partition the file into number of nodes that you have in your cluster

Lets take a look at its first few rows

In [19]:
ratings.rdd.getNumPartitions()

4

In [18]:
ratings.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



We can see that the time stamp are unix (number of seconds since 1 january 1970) 

**I am going to convert timestamp to human readable timestamp**